In [286]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit
import yaml
import pandas as pd
import numpy as np
import joblib
import model_search
import model_lib

ts_cv = TimeSeriesSplit(
    n_splits=5,
    max_train_size=None,
)
f = open("params.yaml", "r")
params = yaml.load(f, Loader=yaml.SafeLoader)
f.close()

In [287]:
def read_data(params):
    x_train = joblib.load(params['DUMP_TRAIN'])
    y_train = joblib.load(params['Y_PATH_TRAIN'])
    x_valid = joblib.load(params['DUMP_VALID'])
    y_valid = joblib.load(params['Y_PATH_VALID'])

    return x_train, y_train, x_valid, y_valid

def training_model():

    #read data
    x_train, y_train, x_valid, y_valid = read_data(params)

    ridge = [Ridge(), model_lib.model_ridge]
    lasso = [Lasso(), model_lib.model_lasso]
    rf = [RandomForestRegressor(), model_lib.model_rf]
    svr = [LinearSVR(), model_lib.model_svr]

    # define model
    model = eval(joblib.load(params['MODEL_NAME']))[0]
    alpa = eval(joblib.load(params['MODEL_NAME']))[1]

    scoring = 'neg_root_mean_squared_error'

    # define search
    search = GridSearchCV(model, alpa(), scoring=scoring, n_jobs=-1, cv=ts_cv)

    # execute search
    result = search.fit(x_train, y_train)

    # summarize result
    print('Best Score: %s' % result.best_score_)
    print('Best Hyperparameters: %s' % result.best_params_)

In [288]:
training_model()

Best Score: -15.79957690352283
Best Hyperparameters: {'alpha': 0.1}
